## Import Python Library
1. You may need to install torch_geometric and the corresponding packages. Please run pip install requirements.txt first
2. If you are installing it for the first time, it will take a while
3. We are using a nvidia GPU for our experiment.

In [1]:
import torch
import os
import math
import numpy as np
import pandas as pd
import yaml
from utils.helper import print_loss_stat, seed_everything, eval_results

In [2]:
from loader import load_data, load_unsuper_data
from model import TLModel
from tqdm import tqdm

### prepare data
1. We have upload the dataset in the google drive: [here](https://drive.google.com/drive/folders/1ps7xXsIONxvUJu9nOvD_9_hl9W9aI3V7?usp=drive_link)
2. We didn't include the dataset into our github repo because of the size issue


### Define the yaml file
+ **Dataset**, Put the data into the datasets
+ **Model**:
    + Input_Dim: is related with the dataset
    + Conv: define the graph Convolution method. Specifically, please choose “TransformerConv” for running the SpaRx model, “GATConv” for the SpaRx-GAT, and “GCNConv” for the SpaRx-GCN model 
    + NUM_HIDDENS: the hidden dims for the first layer and the second layer
    + num_classes: 2 categories, one for the source, another for the target

In [3]:
yaml_file = 'configure_default.yml'
with open(yaml_file) as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

seed_everything(cfg['SEED'])

### train the transfer model
Define the settings in the yaml file:

+ **lr:**: the learning rate
+ **Epochs:** training epochs
+ **Save_path:** the checkpoint path
+ **Momentum** and **Weight_dacay**: parameters for the Adam Optimizer
+ **grad_clip:** clip the gradient when it was too large



In [4]:
def train(cfg):
    # build data
    src_data_root = cfg['DATASET']['Source_data_root']
    src_label_root = cfg['DATASET']['Source_label_root']
    src_adj_root = cfg['DATASET']['Source_adj_root']

    src, src_train_idx, src_test_idx = load_data(root=src_data_root,
                                    adj_root=src_adj_root,
                                    label_root=src_label_root,
                                    num_val=0.2, random_state=cfg['SEED'])
    
    tar_data_root = cfg['DATASET']['target_data_root']
    tar_label_root = cfg['DATASET']['target_label_root']
    tar_adj_root = cfg['DATASET']['target_adj_root']
    tar = load_unsuper_data(root=tar_data_root,
                            adj_root=tar_adj_root,
                            label_root=tar_label_root)

    best_loss = 10000000
    best_f1 = 0
    best_epoch = 0
    # # build up model
    model = TLModel(cfg)
    for epoch in tqdm(range(cfg['TRAIN']['Epochs'])):
        # print('best_F1 is %f in Epoch %d' %(best_f1, best_epoch))
        model.set_input(src, tar)
        model.update_parameters()
        loss_stat = model.get_current_loss()
        loss = loss_stat['overall_loss'].item()

        # print_loss_stat(loss_stat, epoch, cfg['TRAIN']['Epochs'])

        f1 = val(cfg, model, tar)
        print(f1)
        if best_f1 < f1:
            model.save('best_f1')
            best_f1 = f1
            best_epoch = epoch
        if best_loss > loss:
            model.save('best_loss')
            best_loss = loss
            best_epoch = epoch
    model.save("last")
    #src_emb, tar_emb = model.get_emb()
    f1 = val(cfg, model, tar)
    print('f1 is %.4f'%(f1))

In [5]:
@torch.no_grad()
def val(cfg, model, tar):
    model.set_input(tar, tar)
    pred = model.inference()
    pred = pred.detach().cpu()
    label = tar.y.detach().cpu()

    f1 = eval_results(pred, label, cfg['TEST']['Verbose'])
    print("f1 is %.4f"%(f1))
    return f1

### training details
We train SpaRx for 100 epochs. For the demo, we print the performance for each epoch. At the very begining (first 30 epochs), the training model didn't learn the target infos. But the performance grows gradually and the final F1 score achieved to 0.9633.

In [6]:
train(cfg)

Using TransformerConv


  0%|                                                   | 0/100 [00:00<?, ?it/s]


------------------------results----------------------
         0	      1000
      1000	      1000
      acc:	    0.5000
precision:	    0.0000
   recall:	    0.0000
Specificity:	    1.0000
Sensitivity:	    0.0000
F1-measure:	       nan
------------------------------------------------------

f1 is nan
tensor(nan)


  2%|▊                                          | 2/100 [00:01<01:23,  1.17it/s]


------------------------results----------------------
         0	      1000
      1000	      1000
      acc:	    0.5000
precision:	    0.0000
   recall:	    0.0000
Specificity:	    1.0000
Sensitivity:	    0.0000
F1-measure:	       nan
------------------------------------------------------

f1 is nan
tensor(nan)


...

 99%|█████████████████████████████████████████▌| 99/100 [00:38<00:00,  2.62it/s]


------------------------results----------------------
       965	      1000
       954	      1000
      acc:	    0.9595
precision:	    0.9545
   recall:	    0.9650
Specificity:	    0.9540
Sensitivity:	    0.9650
F1-measure:	    0.9597
------------------------------------------------------

f1 is 0.9597
tensor(0.9597)


100%|█████████████████████████████████████████| 100/100 [00:38<00:00,  2.58it/s]


------------------------results----------------------
       969	      1000
       956	      1000
      acc:	    0.9625
precision:	    0.9566
   recall:	    0.9690
Specificity:	    0.9560
Sensitivity:	    0.9690
F1-measure:	    0.9627
------------------------------------------------------

f1 is 0.9627
tensor(0.9627)



------------------------results----------------------
       969	      1000
       956	      1000
      acc:	    0.9625
precision:	    0.9566
   recall:	    0.9690
Specificity:	    0.9560
Sensitivity:	    0.9690
F1-measure:	    0.9627
------------------------------------------------------

f1 is 0.9627
f1 is 0.9627


### Test the Model performance
In this demo, We first test the model with the best f1 score, which was the 969 epoch. After that, we save the embedding for downstream analysis

In [7]:
@torch.no_grad()
def test(cfg):
    tar_data_root = cfg['DATASET']['target_data_root']
    tar_label_root = cfg['DATASET']['target_label_root']
    tar_adj_root = cfg['DATASET']['target_adj_root']
    tar, tar_index = load_unsuper_data(root=tar_data_root,
                            adj_root=tar_adj_root,
                            label_root=tar_label_root, return_index=True)

    model = TLModel(cfg)
    model.load(cfg['TEST']['Save_path'])

    pred = model.set_input(tar, tar)
    pred = model.inference()
    # src_emb, tar_emb = model.get_emb()
    pred = pred.detach().cpu()

    label = tar.y.detach().cpu()
    f1 = eval_results(pred, label)

    # save pred to csv
    True_label = label.numpy()

    df = pd.DataFrame(index=tar_index)
    df['pred'] = pred.numpy()
    df['true_label'] = True_label

    # if not os.path.exists(os.path.join('results', cfg['Name'])):
    #     os.makedirs(os.path.join('results', cfg['Name']))
    
    if not os.path.join(cfg["TEST"]["Pred"]):
        os.makedirs(cfg["TEST"]["Pred"], exist_ok=True)
    
    df.to_csv(os.path.join(cfg['TEST']['Pred'],'prediction.csv'))        
    return f1

In [8]:
test(cfg)

Using TransformerConv


tensor(0.9633)

In [9]:
@torch.no_grad()
def save_emb(cfg):
    src_data_root = cfg['DATASET']['Source_data_root']
    src_label_root = cfg['DATASET']['Source_label_root']
    src_adj_root = cfg['DATASET']['Source_adj_root']

    src, src_index = load_unsuper_data(root=src_data_root,
                            adj_root=src_adj_root,
                            label_root=src_label_root, return_index=True)

    tar_data_root = cfg['DATASET']['target_data_root']
    tar_label_root = cfg['DATASET']['target_label_root']
    tar_adj_root = cfg['DATASET']['target_adj_root']
    tar, tar_index = load_unsuper_data(root=tar_data_root,
                            adj_root=tar_adj_root,
                            label_root=tar_label_root, return_index=True)


    model = TLModel(cfg)
    model.load(cfg['TEST']['Save_path'])

    pred = model.set_input(src, tar)
    src_emb, tar_emb = model.get_emb()

    src_emb = src_emb.detach().cpu().numpy()
    tar_emb = tar_emb.detach().cpu().numpy()

    df = pd.DataFrame(src_emb, index=src_index)
    # if not os.path.exists(os.path.join('results', cfg['Name'])):
    #     os.makedirs(os.path.join('results', cfg['Name']))
    df.to_csv(os.path.join(cfg['TEST']['Pred'],'src_emb.csv'))        

    df = pd.DataFrame(tar_emb, index=tar_index)
    df.to_csv(os.path.join(cfg['TEST']['Pred'],'tar_emb.csv'))        

In [10]:
save_emb(cfg)

Using TransformerConv


### How to choose the model when we don't know the ground truth?
1. In real implementation, we didn't have the ground truth, and therefore we provided another model called best_loss.pth, which had the smallest training loss. This can have overfitting issues and the performance is 0.8924

In [11]:
cfg['TEST']['Save_path'] = "./checkpoint/best_loss.pth"
seed_everything(cfg['SEED'])

In [12]:
print(cfg['TEST']['Save_path'])

./checkpoint/best_loss.pth


In [13]:
test(cfg)

Using TransformerConv


tensor(0.8924)

2. We also present the result from the last epoch after training 100 times. The F1 score in the last layer is 0.9627, which is compariable with the best F1 model (0.9633)

In [14]:
cfg['TEST']['Save_path'] = "./checkpoint/last.pth"
seed_everything(cfg['SEED'])
print(cfg['TEST']['Save_path'])
test(cfg)

./checkpoint/last.pth
Using TransformerConv


tensor(0.9627)